In [7]:
import inspect
import os
import json
import numpy as np
import subprocess
from shutil import make_archive
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers import ChallengeWrapper
import ray
from ray import tune
import ray.rllib.algorithms.ppo as ppo
from ray.tune.registry import register_env

In [8]:
def wrap(env):
    return ChallengeWrapper(env=env, agent_name='Blue')

def env_creator(env_config):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = ChallengeWrapper(env=cyborg, agent_name='Blue')
    return env

register_env("cyborg", env_creator)

In [9]:
# Configuration used to train the model (not sure if needed)
config = ppo.DEFAULT_CONFIG.copy()
config['framework'] = "tf"
config['env'] = 'cyborg'
config['num_gpus'] = 1
config["num_workers"] = 3
config['horizon'] = 1024
config['train_batch_size'] = 1024
config['sgd_minibatch_size'] = 128
config['rollout_fragment_length'] = 100
# config['model'] = {
#     "fcnet_hiddens": [512, 512],
#     "fcnet_activation": "relu"
# }
config['batch_mode'] = "truncate_episodes"
config['lambda'] = 0.95
config['kl_coeff'] = 0.5
config['clip_rewards'] = True
config['clip_param'] = 0.1
config['vf_clip_param'] = 10.0
config['entropy_coeff'] = 0.01
config['vf_share_layers'] = True
# config['num_sgd_iter'] = 10

# Load the model from path
trained_model_path = "C:/Users/Rafael/ray_results/experiment_2022-07-05_11-56-56/experiment_cyborg_18c39_00000_0_2022-07-05_11-56-56/checkpoint_005000/checkpoint-5000"
agent = ppo.PPO(config=config, env="cyborg")
agent.restore(trained_model_path)

2022-07-16 09:01:47,501	WARNING ppo.py:350 -- `train_batch_size` (1024) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=100)! Auto-adjusting `rollout_fragment_length` to 342.
(RolloutWorker pid=28324) 2022-07-16 09:01:57,108	ERROR worker.py:758 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=28324, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001DE96C671C0>)
(RolloutWorker pid=28324)   File "python\ray\_raylet.pyx", line 625, in ray._raylet.execute_task
(RolloutWorker pid=28324)   File "c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-packages\ray\_private\memory_monitor.py", line 162, in raise_if_low_memory
(RolloutWorker pid=28324)     raise RayOutOfMemoryError(
(RolloutWorker pid=28324) ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node DESKTOP-HO3GS2A is used (15.48 

RayOutOfMemoryError: More than 95% of the memory on node DESKTOP-HO3GS2A is used (15.48 / 15.96 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
9448	1.23GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe -m ipykernel_launcher --ip=127.0.0.1 --stdin=9003
24180	1.08GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
8528	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
27372	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
3952	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
23760	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
28012	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
9676	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cybo

(RolloutWorker pid=28324) 24180	1.08GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 8528	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 27372	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 3952	1.07GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 23760	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 28012	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker pid=28324) 9676	0.98GiB	c:\Users\Rafael\Anaconda3\envs\cyborg2\python.exe c:\Users\Rafael\Anaconda3\envs\cyborg2\lib\site-pa
(RolloutWorker p

In [ ]:
MAX_EPS = 100
agent_name = 'Blue'
scenarios = ["Scenario1b"]
steps = [30, 50, 100]
red_agents = [B_lineAgent, RedMeanderAgent, SleepAgent]

for scenario in scenarios:
    path = (str(inspect.getfile(CybORG))[:-10] + '/Shared/Scenarios/{}.yaml').format(scenario)
    print("EVALUATION FOR", scenario)
    for num_steps in steps:
        for red_agent in red_agents:
            cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
            wrapped_cyborg = wrap(cyborg)
            observation = wrapped_cyborg.reset()
            # observation = cyborg.reset().observation
            action_space = wrapped_cyborg.get_action_space(agent_name)
            # action_space = cyborg.get_action_space(agent_name)
            total_reward = []
            actions = []
            for i in range(MAX_EPS):
                r = []
                a = []
                # cyborg.env.env.tracker.render()
                for j in range(num_steps):
                    # action = agent.get_action(observation, action_space)
                    # action, _states = agent.predict(observation)
                    action = agent.compute_single_action(observation) # RLlib 
                    # action, state_out, _ = agent.compute_single_action(observation, state) # RLlib with Attention 
                    observation, rew, done, info = wrapped_cyborg.step(action)
                    # result = cyborg.step(agent_name, action)
                    r.append(rew)
                    # r.append(result.reward)
                    a.append((str(cyborg.get_last_action('Blue')), str(cyborg.get_last_action('Red'))))
                total_reward.append(sum(r))
                actions.append(a)
                # observation = cyborg.reset().observation
                observation = wrapped_cyborg.reset()
            print(f'Average reward for red agent {red_agent.__name__} and steps {num_steps} is: {mean(total_reward)} with a standard deviation of {stdev(total_reward)}')